# Tiny-TLS 1.3 Toy Implementation for COSI107a, Spring 2025, version 0.1

This document contains the implementation of a toy version of TLS 1.3, to be used as material for Brandeis' COSI107a course. The goal is to implement a minimalist version of TLS 1.3 that can communicate with a server using the protocol.

This is a work in progress, and it is expected that changes will be made to the protocol as we move forward


## 1. Import the necessary libraries

Since we're working with TLS in Python, it is helpful to use Python libraries that allow us to pack our data into binary form. TLS protocol specifies exact byte length and format, and we'll be doing a lot of conversion between numbers and bytes. We'll use the Python Struct library for this purpose

We'll also be importing the x25519 curve from the Cryptography library to generate the key used in our messages



In [299]:
import struct

from cryptography.hazmat.primitives.asymmetric import x25519
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf import hkdf
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.hmac import HMAC
from cryptography.hazmat.backends import default_backend

import socket
import time
from urllib.parse import urlparse


import os #for random nonce generation

# 2a. Define some helper functions

TLS represents all of its messages in byte form. As such, some elements (such as the content length) must be converted from integer to big edian, specifically with 2 bytes. We also need to be able to join the distinct elements in a message into a single byte slice


In [300]:
def u16_to_byte(x: int) -> bytes:
    return struct.pack('>H', x) # Use the struct package to pack a number into big-edian 2 bytes

def concatenate(*bufs: bytes) -> bytes: # Concatenate multiple byte slices into one singular byte slice
    return b''.join(bufs)

def print_bytes_as_hex(b: bytes) -> None: # This is to print the bytes as hex strings. Easier to double check this way
    hex_string = ' '.join([f'{x:02x}' for x in b])
    print(hex_string)

# 2b. Define our private key and public key

We can then generate the public key and private key using the x25519 curve from Cryptography


In [301]:
def key_pair() -> bytes:
    private_key = x25519.X25519PrivateKey.generate()
    public_key = private_key.public_key().public_bytes(
    encoding=serialization.Encoding.Raw,
    format=serialization.PublicFormat.Raw
    )
    return private_key, public_key

print(key_pair())


(<cryptography.hazmat.backends.openssl.x25519._X25519PrivateKey object at 0x7fb170ff4e80>, b'@:\xd9J\x11\xb1\x16\x03H\x03C\xd4\xf9_7\x0e\x12\xad\xb2=\xbb6\x84\xac\xaa\xc9\xacTx\xdb\x05`')


Notice how each time we run the code, the public_key is different. This is intended behavior, as it prevents attackers from being able to predict our key pairs

# 3. The Extension Blueprint:

Extensions play a large part in shaping a TLS message. From negotiating supported key group to exchanging keys, all of these are achieved using extensions. Luckily for us, these different extensions have a common blueprint



In [302]:
def extension(id: int, content: bytes) -> bytes:
    return concatenate(
        u16_to_byte(id),                         #The ID of the extension. (e.g 0x0a = Supported Group)
        u16_to_byte(len(content)),               #Length of content
        content                                  #The actual content itself
    )

print_bytes_as_hex(extension(0x0a, bytes([0x00, 0x1d]))) #Example Supported Group extension


00 0a 00 02 00 1d


# 4. The ClientHello

With those building blocks, we can now write our first TLS message: The ClientHello. The ClientHello is always the first message to be sent in a TLS handshake, indicating that the client wants to connect with the server

The ClientHello has these components, in this order:
1. ProtocolVersion (Negotiate which version of TLS we're using)
2. Random Nonce (32 bit, for key generation)
3. Legacy Session ID (For our purposes, we won't be using sessions)
4. Cipher Suites (contains a suite of cipher - how to actually encrypt the key once we have it)
5. Legacy Compression Method (For TLS 1.3, this is null)
6. Extensions 

The code for it is as follows:

In [303]:
private_key, public_key = key_pair()

def client_hello() -> bytes:
    client_random = os.urandom(32)
    def key_share(pubkey: bytes) -> bytes:      # Encode our public key to be sent over the message
        return concatenate(
            u16_to_byte(len(pubkey) + 4),       # +4 represents the 4 extra byte before the pubkey (2 bytes for the x25519, 2 bytes for len of pubkey)
            u16_to_byte(0x1d),                  # 0x1d is the value for x25519 key
            u16_to_byte(len(pubkey)),           
            pubkey
        )
    
    def DNI(domain: str) -> bytes:
        return concatenate(
            u16_to_byte(len(bytes(domain,'utf-8')) + 3),
            bytes([0x00]),
            u16_to_byte(len(bytes(domain,'utf-8'))),
            bytes(domain, 'utf-8')
        )
    
    def extensions() -> bytes: #This intializes the extensions we need in our message
        return concatenate(
            extension(0x00, DNI('www.cloudflare.com')),
            extension(0x0a, bytes([0x00, 0x02, 0x00, 0x1d])), #Supported Group extensions. Currently only contains the x25519 curve
            extension(0x0d, bytes([0x00, 0x12, 0x04, 0x03, 0x08, 0x04, 0x04, 0x01, 0x05, 0x03, 0x08, 0x05, 0x05, 0x01, 0x08, 0x06, 0x06, 0x01, 0x02, 0x01])),
            extension(0x33, key_share(public_key)), #Key Share. Contains the public key generated from the x25519 curve
            extension(0x2b, bytes([0x02, 0x03, 0x04])) #TLS Version. This is how we negotiate TLS 1.3
        )
    
    def handshake() -> bytes: #This constitutes our actual ClientHello message
        return concatenate(
            bytes([0x03, 0x03]),                          # This value is for TLS 1.2. TLS 1.3 must disguise itself as TLS 1.2 to be received, after which it negotiates into TLS 1.3 through the TLS 1.3 extension
            client_random,                             # Random Nonce for key
            bytes([0x00]),                                # Session ID. Empty for our purposes
            bytes([0x00, 0x02, 0x13, 0x02]),              # Cipher Suite. We have a single cipher for our cipher suite (SHA384)
                                                          # I'm aware that there are 2 SHA384 ciphers: AES and CHACHA. I've included one here. Not sure
                                                          # if we need the other one or not
            bytes([0x01, 0x00]),                          # Compression Method. Empty for our purposes
            u16_to_byte(len(extensions())),
            extensions()
        )
    
    return concatenate(                                 #Include record layers for TLS 1.3 to complete message
        bytes([0x16, 0x03, 0x01]),
        u16_to_byte(len(handshake()) + 4),
        bytes([0x01,0x00]),
        u16_to_byte(len(handshake())),
        handshake()
    ), client_random

client_hello_msg, client_random = client_hello()
print_bytes_as_hex(client_hello_msg)
client_hello_no_record = client_hello_msg[5:] #This is to calculate the transcript hash


## To check the validity of the ClientHello, copy paste the ClientHello into the openSSL_debug.py script and run it. Server should give back a response

16 03 01 00 9b 01 00 00 97 03 03 ce 0e 4b 66 4a 89 52 23 f3 48 6e 8b 7d 3d 93 28 f6 51 75 04 21 46 ec e5 9f 4f fd 37 c9 65 d0 68 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 cb b0 78 1e 41 f9 db 91 66 91 14 94 5e 64 51 44 b2 c6 52 b2 4f 0c 61 69 7b 80 69 ba fc 2c 27 46 00 2b 00 03 02 03 04


# 5. Sending the ClientHello:
Since we now have the ClientHello, the next step would be to send it over to a public domain and see if the server responds. For this part, we'll be using www.cloudbase.com, as it supports TLS 1.3, and the cipher suite matches with our clientHello.


In [304]:
server_hello_no_record = b""
encrypted_payload = b""



def parse_tls_record(data):
    #Parse a TLS record and return its components
    if len(data) < 5:  # Minimum TLS record length
        return None, None, None, data
    
    content_type = data[0] #Parse the record header to see which type of record this is.
    if (data[1] == 3) and (data[2] == 3):
        version = "1.2/1.3"
    length = int.from_bytes(data[3:5], byteorder='big')
    
    if len(data) < length + 5:
        return None, None, None, data
        
    if content_type == 0x17:
        payload = data[:5+length]  # Include the header for application data. We need the header for the decryption algorithm
        remaining_data = data[5+length:]
    else:
        payload = data[5:5+length]  # Original behavior for other types
        remaining_data = data[5+length:]
    
    content_type_names = { #Dictionary indicating the types of TLS records we may receive
        0x14: "Change Cipher Spec",
        0x15: "Alert",
        0x16: "Handshake",
        0x17: "Application Data"
    }
    
    type_name = content_type_names.get(content_type, f"Unknown ({content_type})")
    
    return type_name, version, payload, remaining_data

def parse_handshake_message(data): #This is exclusive for Handshake messages. We'll primarily use this to identify serverHellos.
    """Parse a TLS handshake message."""
    if len(data) < 4:
        return None, None, None
        
    msg_type = data[0]
    length = int.from_bytes(data[1:4], byteorder='big')
    
    handshake_types = {
        0x00: "Hello Request",
        0x01: "Client Hello",
        0x02: "Server Hello",
        0x0b: "Certificate",
        0x0c: "Server Key Exchange",
        0x0d: "Certificate Request",
        0x0e: "Server Hello Done",
        0x0f: "Certificate Verify",
        0x10: "Client Key Exchange",
        0x14: "Finished",
        0x08: "Encrypted Extensions",  # Added TLS 1.3 message types
        0x13: "Certificate Status"
    }
    
    type_name = handshake_types.get(msg_type, f"Unknown ({msg_type})")
    if len(data) >= 4+length:
        payload = data[:4+length]
    
    return type_name, length, payload

def format_bytes(data):
    """Format bytes as a hexadecimal string with ASCII representation."""
    hex_dump = ' '.join(f'{b:02x}' for b in data)
    return f"Hex: {hex_dump}"

def receive_all_data(sock, timeout=5):
    """Receive all data from the socket until timeout or connection close."""
    buffer = b""
    sock.settimeout(timeout)
    start_time = time.time()
    
    while True:
        try:
            chunk = sock.recv(16384)
            if not chunk:  # Connection closed by server
                break
            buffer += chunk
            
            # Check timeout
            if time.time() - start_time > timeout:
                break
                
        except socket.timeout:
            break
            
    return buffer
# Parse the URL to get the hostname
parsed_url = urlparse('https://www.cloudflare.com/')
host = parsed_url.hostname
port = 443
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.settimeout(5)
print(f"Attempting to connect to {host}:{port}")
s.connect((host, port))
print("Connected to server.")

# Your existing client_hello
try:
    
    bytes_sent = s.sendall(client_hello_msg)
    print(f"Sent ClientHello ({len(client_hello_msg)} bytes)")
    
    print("\nWaiting for server response...")
    try:
        buffer = receive_all_data(s)
        print(f"\nReceived {len(buffer)} new bytes")
        print(print_bytes_as_hex(buffer))


        
        # Process all complete TLS records in the buffer
        while buffer:
            record_type, version, payload, remaining = parse_tls_record(buffer)
            if record_type is None:  # Incomplete record
                break
                
            buffer = remaining
            print("\n" + "="*50)
            print(f"Record Type: {record_type}")
            print(f"TLS Version: {version}")
            
            if record_type == "Handshake":
                msg_type, msg_length, msg_payload = parse_handshake_message(payload)
                if msg_type:
                    print(f"Handshake Type: {msg_type}")
                    print(f"Message Length: {msg_length}")
                    if msg_payload:
                        print("\nMessage Payload:")
                        print(format_bytes(msg_payload))
                        server_hello_no_record = msg_payload
            else:
                print("\nRecord Payload:")
                if record_type == "Application Data":
                    encrypted_payload = payload
                print(format_bytes(payload))
            
    except socket.timeout:
        print("Timeout while waiting for server response")
            
except ConnectionRefusedError:
    print("Connection refused. Is the OpenSSL server running?")
except socket.timeout:
    print("Connection attempt timed out")
except Exception as e:
    print(f"Error: {e}")

print("\nExtracted Data:")
if server_hello_no_record:
    print("\nServer Hello (without record header):")
    print(print_bytes_as_hex(server_hello_no_record))
if encrypted_payload:
    print("\nEncrypted Payload (with record header):")
    print(print_bytes_as_hex(encrypted_payload))

Attempting to connect to www.cloudflare.com:443
Connected to server.
Sent ClientHello (160 bytes)

Waiting for server response...

Received 2813 new bytes
16 03 03 00 5a 02 00 00 56 03 03 7b 63 f1 74 91 ca f8 49 95 c2 5c 66 e5 db 30 28 88 f1 6a f3 5d be 7e 59 85 c2 cb 86 59 37 67 7a 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 31 2e d9 ac a1 29 5a cc b9 45 47 60 0d 73 66 52 bc 91 64 00 61 df 89 90 95 eb 0a 2b 60 2a 01 0b 00 2b 00 02 03 04 14 03 03 00 01 01 17 03 03 0a 93 b8 f1 1e 2c 9d 6b dd 2a 24 03 ae 63 52 11 a9 07 09 0c b5 6f b3 80 f0 69 5d d2 26 33 bb 43 61 d8 07 ba ab bd 40 ad bd f0 74 3e 21 8d 40 b3 64 89 d7 af 1a e3 fa 15 3c 2f 71 e1 cf e8 0a 4a b6 29 6a c6 f8 55 5d da e1 b1 4c cc 3d 6b f3 a2 08 c7 01 fe 46 20 30 bf 99 f2 d7 c1 02 6c 38 24 a9 90 25 3f 82 e7 06 54 c6 93 6f 86 41 f8 5a 3d d0 8a da 34 e5 cc a0 59 25 6a 05 a7 03 b7 6d ce 6c 68 3c 71 32 53 01 c2 b4 85 5a e1 f9 3d fc 59 2e 45 7f 80 ef 41 c3 9e 07 7b 9b 91 93 15 09 94 09 ef 6a 6b 34 a8 e0 13 50 75 6a 0a d6 2c 15 7d ac 79

# 6. Parsing the ServerHello

Once the ClientHello is sent, the server responds with its own message. This message contains the ServerHello, the CipherChangeSpec, and the encrypted payload. Assuming that our ClientHello message is configured correctly, the server will respond with its chosen cipher suite and its own key. 

We are particularly interested in the server's key and server random, which we use to establish cryptographic parameters. We can then extract these properties and start the key calculation process

# 6a. Creating a Parser class 

The ServerHello message isn't always of the same size. Certain elements like the sessionID, as well as the content of extensions, may have variable length depending on the message itself. That's why we can't just extract the information based on indexes alone. We need a parser that would keep track of what element we're at and how many bytes we have to skip forward


In [305]:

class Parser:
    def __init__(self, data: bytes) -> None:
        self.data = data
        self.cursor = 0
    
    def skip(self, position: int) -> None:
        self.cursor += position

    def read(self, position: int) -> bytes:
        result = self.data[self.cursor : self.cursor + position]
        self.cursor += position
        return result 
    
    def read_uint8_prefixed(self) -> bytes:                           #Most extensions with variable lengths have bytes that denote their length. We can use this to skip forward appropriately 
        length = self.data[self.cursor]
        self.cursor += 1
        result = self.data[self.cursor : self.cursor + length]
        self.cursor += length
        return result

    def read_uint16_prefixed(self) -> bytes:
        length = int.from_bytes(self.data[self.cursor:self.cursor + 2], 'big') #Since some lengths are represented with 2 bytes, we need to convert them 
        self.cursor += 2
        result = self.data[self.cursor:self.cursor + length]
        self.cursor += length
        return result


# 6b. Parsing the ServerHello

With our Parser class, we can now parse the ServerHello to extract the ServerRandom and the public key. Keep in mind that we are only dealing with 1 cipher suite, 1 cryptographic and as such expects only 1 public key. A full version of TLS 1.3 will be much more complex

In [306]:
def server_hello_parser(msg: bytes):
    parser = Parser(msg)
    parser.skip(4)                                  #Skip HandShake Header
    parser.skip(2)                                  #Skip Server Version
    server_random = parser.read(32)                 #Get the serverRandom
    parser.read_uint8_prefixed()                    #Skip SessionID
    parser.skip(2)                                  #Skip Cipher Suite
    parser.skip(1)                                  #Skip Compression Method
    public_key = None
    extensions = parser.read_uint16_prefixed()
    extension_reader = Parser(extensions)
    while(extension_reader.cursor < len(extensions)):
        extension_type = extension_reader.read(2)
        extension_data = extension_reader.read_uint16_prefixed()
        if (extension_type == b'\x00\x33'):
            data = Parser(extension_data)
            data.skip(2)
            public_key = data.read_uint16_prefixed()
    return server_random, public_key


#Pre-generated server_hello_msg, used for debugging purposes
#server_hello_msg = b'\x02\x00\x00\x76\x03\x03\x70\x71\x72\x73\x74\x75\x76\x77\x78\x79\x7a\x7b\x7c\x7d\x7e\x7f\x80\x81\x82\x83\x84\x85\x86\x87\x88\x89\x8a\x8b\x8c\x8d\x8e\x8f\x20\xe0\xe1\xe2\xe3\xe4\xe5\xe6\xe7\xe8\xe9\xea\xeb\xec\xed\xee\xef\xf0\xf1\xf2\xf3\xf4\xf5\xf6\xf7\xf8\xf9\xfa\xfb\xfc\xfd\xfe\xff\x13\x02\x00\x00\x2e\x00\x2b\x00\x02\x03\x04\x00\x33\x00\x24\x00\x1d\x00\x20\x9f\xd7\xad\x6d\xcf\xf4\x29\x8d\xd3\xf9\x6d\x5b\x1b\x2a\xf9\x10\xa0\x53\x5b\x14\x88\xd7\xf8\xfa\xbb\x34\x9a\x98\x28\x80\xb6\x15'
#Run the script with the pre-generated server hello and compare with https://tls13.xargs.org/#server-handshake-keys-calc to see if the script is working
#Current serverHello generated from www.cloudbase.com. Right now I'm copy-pasting from the terminal when I run the debug.py script. Also parser is ignoring the record header.
server_random, public_key = server_hello_parser(server_hello_no_record)
print_bytes_as_hex(server_random)
print_bytes_as_hex(public_key)


           

        
        
        


7b 63 f1 74 91 ca f8 49 95 c2 5c 66 e5 db 30 28 88 f1 6a f3 5d be 7e 59 85 c2 cb 86 59 37 67 7a
31 2e d9 ac a1 29 5a cc b9 45 47 60 0d 73 66 52 bc 91 64 00 61 df 89 90 95 eb 0a 2b 60 2a 01 0b


# 7. Key Derivation

Now that we have our private key and the server has sent back their public key, we can now encrypt our data to the server via key calculations. We can break down the key calculations into a few steps

# 7a. Transcript Hash

Transcript Hash refers to the hash of the ClientHello and the ServerHello messages. The idea is to associate these messages with the keys we are about to derive: These keys work with these messages. That way, even if an attacker somehow got hold of our keys, it would not work with whatever messages they try to send



In [307]:
def transcript_hash(client: bytes, server: bytes) -> bytes:
    digest = hashes.Hash(hashes.SHA384()) # We'll be using SHA384 to create the hash for our messages.
    digest.update(client)     # Note that both the ClientHello and ServerHello is used here
    digest.update(server)
    transcript_hash = digest.finalize()
    return transcript_hash

print_bytes_as_hex(client_hello_no_record)       #This is the clientHello message we generated from our code
print_bytes_as_hex(server_hello_no_record)       #Right now, this server_hello is pre-generated for testing purposes. We'll be parsing an actual server hello when the entire thing is ready

hello_hash = transcript_hash(client_hello_no_record, server_hello_no_record)
print_bytes_as_hex(hello_hash)

01 00 00 97 03 03 ce 0e 4b 66 4a 89 52 23 f3 48 6e 8b 7d 3d 93 28 f6 51 75 04 21 46 ec e5 9f 4f fd 37 c9 65 d0 68 00 00 02 13 02 01 00 00 6c 00 00 00 17 00 15 00 00 12 77 77 77 2e 63 6c 6f 75 64 66 6c 61 72 65 2e 63 6f 6d 00 0a 00 04 00 02 00 1d 00 0d 00 14 00 12 04 03 08 04 04 01 05 03 08 05 05 01 08 06 06 01 02 01 00 33 00 26 00 24 00 1d 00 20 cb b0 78 1e 41 f9 db 91 66 91 14 94 5e 64 51 44 b2 c6 52 b2 4f 0c 61 69 7b 80 69 ba fc 2c 27 46 00 2b 00 03 02 03 04
02 00 00 56 03 03 7b 63 f1 74 91 ca f8 49 95 c2 5c 66 e5 db 30 28 88 f1 6a f3 5d be 7e 59 85 c2 cb 86 59 37 67 7a 00 13 02 00 00 2e 00 33 00 24 00 1d 00 20 31 2e d9 ac a1 29 5a cc b9 45 47 60 0d 73 66 52 bc 91 64 00 61 df 89 90 95 eb 0a 2b 60 2a 01 0b 00 2b 00 02 03 04
ce bf f6 c1 97 c2 13 c6 bd bc ee 7d c1 fb 2c cd 95 a5 b8 ac 0e dc 1b 5f 7f 21 5c f7 25 85 54 e2 f9 e0 63 7b 6f 00 a1 08 ca c7 39 17 cb 6d cc 4c


# 7b. Shared secret
The idea of the key exchange is that given the private key and the public key of the other party, both client and server can perform calculations to arrive at the same number. Essentially, client private x server public = server private x client public. This is called the shared secret. We'll perform the calculations on our side while the server does it on theirs.


In [308]:
# Our public key right now is raw bytes. Our private key, on the other hand, is a x25519PrivateKey object. The most convenient way to calculate the 
# shared secret is to turn the public key into a x25519PublicKey object and leverage the built in methods to generate the shared secret

server_public_key = x25519.X25519PublicKey.from_public_bytes(public_key) 
shared_secret = private_key.exchange(server_public_key)
print_bytes_as_hex(shared_secret)



96 70 f3 6b bc f8 36 8a cf 12 1e 5e 47 60 fc 15 09 e1 4c 62 a0 ed cd 7c 84 fd ab 26 3a d7 bb 39


# 7c. Early Secret, Derived Secret, Handshake Secret
We then go on to generate the early secret, the derived secret and finally the handshake secret. 

- Early secret is for 0-RTT data (essentially data sent with pre-shared keys). Since we're not using PSKs, this is a bunch of 0 bytes.
- Derived secret is mainly used as salt, mixed in with the early secret to get the handshake key. This way, even if an attacker has the early secret, it's hard to determine the handshake key

Each of these secrets have a "label" associated with them to identify their use, usually with the format "tls13 + ID of the component". 

* Note 1: We can pass in the label directly into the HKDF info parameter, or we can make a method that will append the TLS13 infront and add the necessary bytes for length, after which it is passed into the infor parameter. To avoid hardcoding, I went with the second options. If this example works, we can go back and try hardcoding the labels to see if it affects anything


In [309]:
## REMEMBER TO CHANGE IT BACK TO SHA256 AFTER TESTING. Algorithm goes back to SHA256, Length goes back to 32.


# UNCOMMENT THIS FOR TESTING. SHOULD WORK FOR SHA384 IMPLEMENTATIONS:

shared_secret = bytes.fromhex('df4a291baa1eb7cfa6934b29b474baad2697e29f1f920dcc77c8a0a088447624')
hello_hash = bytes.fromhex('e05f64fcd082bdb0dce473adf669c2769f257a1c75a51b7887468b5e0e7a7de4f4d34555112077f16e079019d5a845bd')

# expected result
# handshake secret: bdbbe8757494bef20de932598294ea65b5e6bf6dc5c02a960a2de2eaa9b07c929078d2caa09362.31c38d1725f179d299
# client secret: db89d2d6df0e84fed74a2288f8fd4d0959f790ff23946cdf4c26d85e51bebd42ae184501972f8d30c4a3e4a3693d0ef0
# server secret: 23323da031634b241dd37d61032b62a4f450584d1f7f47983ba2f7cc0cdcc39a68f481f2b019f9403a3051908a5d1622
# client handshake key: 1135b4826a9a70257e5a391ad93093dfd7c4214812f493b3e3daae1eb2b1ac69
# client handshake iv: 4256d2e0e88babdd05eb2f27
# server handshake key: 9f13575ce3f8cfc1df64a77ceaffe89700b492ad31b4fab01c4792be1b266b7f
# server handshake iv: 9563bc8b590f671f488d2da3


# TLS specs make use of 2 methods: HKDF-Extract and HKDF-Expand. HKDF-Extract can be found in hkdf.HKDF()_extract(). HKDF-Expand is hkdf.HKDFExpand()


#Define our own hkdf_expand_label. This is basically calling HKDF to derive secrets, with an added step of configuring the labels associated with the component.

def hkdf_expand_label(secret: bytes, label: str, context: bytes, length: int) -> bytes:
    # Construct the HkdfLabel as specified in TLS 1.3
    label = b"tls13 " + label.encode('utf-8')
    hkdf_label = (
        struct.pack("!H", length) +  # 2 bytes for length
        struct.pack("!B", len(label)) + label +  # 1 byte for label length + label
        struct.pack("!B", len(context)) + context  # 1 byte for context length + context
    )
    
    # Use HKDF-Expand
    encryption = hkdf.HKDFExpand(
        algorithm=hashes.SHA384(),
        length=length,
        info=hkdf_label,
    )
    
    return encryption.derive(secret)


early_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(), #We negotiated SHA256 in our cipher suite
    length=48,
    salt=b'\x00',
    info=b'\x00'
)._extract(b'\x00' * 48) #The first input key is simply 00

empty_hash = hashes.Hash(hashes.SHA384())
empty_hash.update(b"")  # Empty string
empty_hash = empty_hash.finalize()

# We then derive another secret called the derived secret from the early secret
derived_secret = hkdf_expand_label(
    secret = early_secret,
    label="derived",
    context=empty_hash,
    length=48
)

# The derived secret is mixed into the early secret to get the handshake secret
handshake_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(),
    length=48,
    salt=derived_secret,  # derived_secret is used as salt
    info=None
)._extract(shared_secret)

# 7d. Traffic Secrets and Key derivation

Now that we have our handshake secret, we can now move on to derive the traffic secrets, and subsequently the keys and IVs for both the client and the server

In [310]:
# Deriving the secrets and subsequent keys + IVs.
client_secret = hkdf_expand_label(
    secret = handshake_secret, 
    label = "c hs traffic", 
    context = hello_hash, 
    length = 48
)

server_secret = hkdf_expand_label(
    secret = handshake_secret,
    label = "s hs traffic",
    context = hello_hash,
    length = 48
)


client_handshake_key = hkdf_expand_label(
    secret = client_secret,
    label = "key",
    context = b"",
    length = 32
)

server_handshake_key = hkdf_expand_label(
    secret = server_secret,
    label = "key",
    context = b"",
    length = 32
)

client_handshake_iv = hkdf_expand_label(
    secret = client_secret,
    label = "iv",
    context = b"",
    length = 12
)

server_handshake_iv = hkdf_expand_label(
    secret = server_secret,
    label = "iv",
    context = b"",
    length = 12
)




print("handshake secret: " + handshake_secret.hex())
print("client secret: " + client_secret.hex())
print("server secret: " + server_secret.hex())
print("client handshake key: " + client_handshake_key.hex())
print("client handshake iv: "+ client_handshake_iv.hex())
print("server handshake key: " + server_handshake_key.hex())
print("server handshake iv: " + server_handshake_iv.hex())


handshake secret: bdbbe8757494bef20de932598294ea65b5e6bf6dc5c02a960a2de2eaa9b07c929078d2caa0936231c38d1725f179d299
client secret: db89d2d6df0e84fed74a2288f8fd4d0959f790ff23946cdf4c26d85e51bebd42ae184501972f8d30c4a3e4a3693d0ef0
server secret: 23323da031634b241dd37d61032b62a4f450584d1f7f47983ba2f7cc0cdcc39a68f481f2b019f9403a3051908a5d1622
client handshake key: 1135b4826a9a70257e5a391ad93093dfd7c4214812f493b3e3daae1eb2b1ac69
client handshake iv: 4256d2e0e88babdd05eb2f27
server handshake key: 9f13575ce3f8cfc1df64a77ceaffe89700b492ad31b4fab01c4792be1b266b7f
server handshake iv: 9563bc8b590f671f488d2da3


## 8. Decryption

Now that we have the keys and IVs, the next step would be to decrypt the encrypted records. When sending the ClientHello, the server sends back a massive binary dump, including the ServerHello, the change cipher specs, and the actual payload, which is encrypted. We will use the keys and ivs generated to decrypt these records.

Normally, you'll see a bunch of records, each with its own content, from the server encrypted extension, certificates, etc. However, Cloudbase has aggregated all of these into a single record. What we're really interested in is the server's finished message, which

In [311]:
def decrypt(key, iv, wrapper):
    # Split the wrapper into additional authenticated data (AAD) and ciphertext
    additional = wrapper[:5]
    ciphertext = wrapper[5:]
    
    # Initialize AES-GCM with the provided key
    aesgcm = AESGCM(key)
    
    # Decrypt the ciphertext
    try:
        plaintext = aesgcm.decrypt(iv, ciphertext, additional)
    except Exception as e:
        raise ValueError(f"Decryption failed: {e}")
    
    return plaintext

def find_finished_message(payload: bytes) -> int:
   # Look for Finished message header: 14 00 00 30 (for SHA384)
   finished_header = bytes([0x14, 0x00, 0x00, 0x30])
   return payload.find(finished_header)



# Right now I'm doing a lot of copy pasting. I run the openSSL_debug script to retrieve the server's response, and then replace the serverHello, as well as the encrypted record under this line. Managed to decrypt the thing.

payload = decrypt(server_handshake_key, server_handshake_iv, encrypted_payload)
print_bytes_as_hex(payload)

# Get index where Finished message starts
finished_index = find_finished_message(payload)
print(finished_index)

# Get everything before the Finished message
server_handshake_no_finished = payload[:finished_index] if finished_index != -1 else payload
server_handshake_finished_message = payload[finished_index:]

print("Server handshake: " + server_handshake_no_finished.hex())
print("Finished message: " + server_handshake_finished_message.hex())



ValueError: Decryption failed: 

# 9. Application keys:
The application keys are nearly the last step when it comes to finishing the handshake. The handshake keys were there decrypt the server's response, containing encrypted extensions, certificates, etc. Once we decrypt the encryption, we then use them to generate another set of keys, called application keys. This will be the main keys we use to communicate with the server. The code themselves are basically the same as the ones used to generate handshake hash

In [ ]:
##!!!!!!!! Potential bug in generating key hash
def application_key_hash(client: bytes, server: bytes, payload: bytes) -> bytes:
    digest = hashes.Hash(hashes.SHA384()) # We'll be using SHA384 to create the hash for our messages.
    digest.update(client)     # Note that both the ClientHello and ServerHello is used here
    digest.update(server)
    digest.update(payload)
    application_key_hash = digest.finalize()
    return application_key_hash

application_context = application_key_hash(client_hello_no_record, server_hello_no_record, server_handshake_no_finished)
print_bytes_as_hex(application_context)

#TestValues. Uncomment this and compare with https://tls13.xargs.org/#server-handshake-keys-calc
handshake_secret = bytes.fromhex("bdbbe8757494bef20de932598294ea65b5e6bf6dc5c02a960a2de2eaa9b07c929078d2caa0936231c38d1725f179d299")
application_context = bytes.fromhex("fa6800169a6baac19159524fa7b9721b41be3c9db6f3f93fa5ff7e3db3ece204d2b456c51046e40ec5312c55a86126f5")

zeros = bytes([0] * 32)
derived_secret = hkdf_expand_label(
    secret = handshake_secret,
    label="derived",
    context=empty_hash,
    length=48
)
master_secret = hkdf.HKDF(
    algorithm=hashes.SHA384(),
    length=48,
    salt=derived_secret,  # derived_secret is used as salt
    info=None
)._extract(b'\x00' * 48)


client_app_secret = hkdf_expand_label(
    secret = master_secret,
    label="c ap traffic",
    context = application_context,
    length = 48
)

server_app_secret = hkdf_expand_label(
    secret = master_secret,
    label = "s ap traffic",
    context = application_context,
    length = 48
)


client_app_key = hkdf_expand_label(
    secret = client_app_secret,
    label = "key",
    context = b"",
    length = 32
)

server_app_key = hkdf_expand_label(
    secret = server_app_secret,
    label = "key",
    context = b"",
    length = 32
)

client_app_iv = hkdf_expand_label(
    secret = client_app_secret,
    label = "iv",
    context = b"",
    length = 12
)

server_app_iv = hkdf_expand_label(
    secret = server_app_secret,
    label = "iv",
    context = b"",
    length = 12
)

print("client secret: " + client_secret.hex())
print("server secret: " + server_secret.hex())
print("client app key: " + client_app_key.hex())
print("client app iv: "+ client_app_iv.hex())
print("server app key: " + server_app_key.hex())
print("server app iv: " + server_app_iv.hex())


f7 0c c8 8e 34 61 59 14 4f b9 3d 8e 9b 68 4d 0e e3 72 b6 4c e0 f9 df 63 16 d3 f6 fc 97 a2 92 f0 a5 bc 79 1a 98 8c f2 5b 43 e7 56 7a f0 a4 21 aa
client secret: 8de3d8552140e9498e9e49fa48b84b1abcb31f760968915d737ed78a33925ed4cad9160bea8a50347f3de5012539a9c3
server secret: 02b1c99dc83a20014712d2581f5d80a490d9716f729d1e20d9b8953e934770859a86083a7309ef949ba3021b851a243b
client handshake key: de2f4c7672723a692319873e5c227606691a32d1c59d8b9f51dbb9352e9ca9cc
client handshake iv: bb007956f474b25de902432f
server handshake key: 01f78623f17e3edcc09e944027ba3218d57c8e0db93cd3ac419309274700ac27
server handshake iv: 196a750b0c5049c0cc51a541


# 10. Generate the finished message
The last step would be to generate the Finished message and send it to the server. After that, we're finished with the handshake, and can start sending over HTTP requests

In [ ]:
def calculate_verify_data(client_handshake_secret: bytes, client_hello: bytes, server_hello: bytes, server_handshake: bytes) -> bytes:
    # First derive the finished key using HKDF-Expand-Label
    finished_key = hkdf_expand_label(
        secret=client_handshake_secret, 
        label="finished", 
        context=b"", 
        length=48  # SHA-384 output size
    )
    
    # Calculate the transcript hash using SHA-384
    transcript = client_hello + server_hello + server_handshake
    digest = hashes.Hash(hashes.SHA384())
    digest.update(transcript)
    finished_hash = digest.finalize()
    
    # Create HMAC of the transcript hash using finished_key
    h = HMAC(finished_key, hashes.SHA384())
    h.update(finished_hash)
    return h.finalize()

def create_client_finished_message(client_handshake_key: bytes, client_handshake_iv: bytes, client_handshake_secret: bytes, 
                                   client_hello: bytes, server_hello: bytes, server_handshake: bytes) -> bytes:
    # Get the verify_data using the function above
    verify_data = calculate_verify_data(client_handshake_secret, client_hello, server_hello, server_handshake)
    
    # Create the Finished message structure
    msg = bytes([0x14, 0x00, 0x00]) + len(verify_data).to_bytes(1, 'big')  # Type and length
    msg += verify_data                      # The HMAC
    msg += bytes([0x16])                    # Handshake message type
    
    # Record layer header as AAD
    print_bytes_as_hex(msg)
    msg_len = len(msg) + 16  # AES-GCM adds a 16-byte authentication tag
    additional = bytes([0x17, 0x03, 0x03]) + msg_len.to_bytes(2, 'big')
    
    # Encrypt using AES-GCM
    aesgcm = AESGCM(client_handshake_key)
    encrypted = aesgcm.encrypt(client_handshake_iv, msg, additional)
    
    # Return the full TLS record
    return additional + encrypted

verify_data = calculate_verify_data(client_secret, client_hello_no_record, server_hello_no_record, server_handshake_no_finished)
print_bytes_as_hex(verify_data)
finished_message = create_client_finished_message(
    client_handshake_key=client_handshake_key,
    client_handshake_iv=client_handshake_iv,
    client_handshake_secret= client_secret,
    client_hello= client_hello_no_record,
    server_hello= server_hello_no_record,
    server_handshake = server_handshake_no_finished
    )
print_bytes_as_hex(finished_message)

# Usage example:
# finished_message = create_client_finished_message(
#     client_handshake_key,
#     client_handshake_iv,
#     client_handshake_secret,
#     client_hello,
#     server_hello,
#     server_handshake
# )

e2 3d 2b b6 91 f2 d5 cc 80 e7 12 63 0d 1c 44 24 69 bf 27 4c fb 21 8e 4c 76 b2 77 79 d0 6c df e3 14 84 41 ac 7e a1 89 e1 3e 85 d5 f2 19 13 48 53
14 00 00 30 e2 3d 2b b6 91 f2 d5 cc 80 e7 12 63 0d 1c 44 24 69 bf 27 4c fb 21 8e 4c 76 b2 77 79 d0 6c df e3 14 84 41 ac 7e a1 89 e1 3e 85 d5 f2 19 13 48 53 16
17 03 03 00 45 4f 97 31 f9 e0 bd bd 63 50 7e e7 6b ef 21 da 74 6f 14 62 65 0a ba 1a f4 f4 24 e4 00 30 e3 b0 93 06 dc 2c 28 22 06 23 44 48 6b 0e 07 fd f1 f3 67 11 32 22 d6 d1 8b 52 ca 37 a1 76 70 05 b0 57 6f 7c 7c 77 e9 50


In [ ]:
s.sendall(finished_message)
print(f"Sent Client Finished message ({len(finished_message)} bytes)")

# Receive response after Finished
response = receive_all_data(s)
print(f"Received response after Finished: {len(response)} bytes")
print_bytes_as_hex(response)





Sent Client Finished message (74 bytes)
Received response after Finished: 24 bytes
17 03 03 00 13 b2 4c 61 d7 88 e9 2d 4e 83 06 b8 2b e4 f5 84 77 15 e5 62


# 11. Decrypting the handshake response:


In [ ]:
sequence_number = 0
nonce = bytes([x ^ y for x, y in zip(
    server_app_iv,
    sequence_number.to_bytes(12, byteorder='big')
)])

additional_data = response[:5]
ciphertext = response[5:]

print("Response:", response.hex())
print("Additional data:", additional_data.hex())
print("Ciphertext:", ciphertext.hex())
print("Server app key:", server_app_key.hex())
print("Server app IV:", server_app_iv.hex())
print("Nonce:", nonce.hex())

aesgcm = AESGCM(server_app_key)
plaintext = aesgcm.decrypt(nonce, ciphertext, additional_data)

Response: 1703030013b24c61d788e92d4e8306b82be4f5847715e562
Additional data: 1703030013
Ciphertext: b24c61d788e92d4e8306b82be4f5847715e562
Server app key: 01f78623f17e3edcc09e944027ba3218d57c8e0db93cd3ac419309274700ac27
Server app IV: 196a750b0c5049c0cc51a541
Nonce: 196a750b0c5049c0cc51a541


InvalidTag: 